In [ ]:
import os
import shutil
from pathlib import Path
import yaml

from ultralytics import YOLO
from ultralytics.data.annotator import auto_annotate
from segment_anything import sam_model_registry
from roboflow import Roboflow

## Config

In [ ]:
model_dir = "../../models"
detection_dataset_dir = "../../data/detection_dataset"
segmentation_dataset_dir = "../../data/segmentation_dataset"

detection_dataset_root = Path(detection_dataset_dir).resolve()
segmentation_dataset_root = Path(segmentation_dataset_dir).resolve()

detection_yaml = f"{detection_dataset_dir}/data.yaml"
segmentation_yaml = f"{segmentation_dataset_dir}/data.yaml"

# The path where segment anything model will be downloaded to
sam_path = f"{model_dir}/sam_b.pt"

In [ ]:
detection_model_config = {
    "model_name": "yolo11n.pt",
    "project_dir": "./runs/detect",
    "run_name": "train",
    "tuned_model_name": "yolo_det.pt",
}

detection_model_config["tuned_model_path"] = f"{model_dir}/{detection_model_config['tuned_model_name']}"

In [ ]:
segmentation_model_config = {
    "model_name": "yolo11n-seg.pt",
    "project_dir": "./runs/segment",
    "run_name": "train",
    "tuned_model_name": "yolo_seg.pt",
}

segmentation_model_config["tuned_model_path"] = f"{model_dir}/{segmentation_model_config['tuned_model_name']}"

## Download the dataset

In [ ]:
roboflow_api_key = os.environ.get("api_key")

In [ ]:
rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace("roboflow-gw7yv").project("vehicles-openimages")
version = project.version(1)
dataset = version.download(model_format="yolov11", location=detection_dataset_dir, overwrite=True)

### Fix yaml paths to absolute instead of relative

In [ ]:
with open(detection_yaml, "r") as f:
    data = yaml.safe_load(f)

for split, dir in [("train", "train"), ("val", "valid"), ("test", "test")]:
    if split in data:
        data[split] = str(detection_dataset_root / dir / "images")

with open(detection_yaml, "w") as f:
    yaml.dump(data, f)

## Fine-tune the detection model

In [ ]:
# Download pretrained model
model = YOLO(detection_model_config["model_name"])

In [ ]:
model.train(
    data=detection_yaml, 
    epochs=100, 
    batch=8, 
    project=detection_model_config["project_dir"], 
    name=detection_model_config["run_name"]
)

In [ ]:
# Save best model in "models" folder
shutil.copy(f"{detection_model_config["project_dir"]}/{detection_model_config["run_name"]}/weights/best.pt", detection_model_config["tuned_model_path"])

## Anotate the detection dataset for segmentation task

In [ ]:
# Copy detection dataset to use it as a base for segmentation
shutil.copytree(detection_dataset_dir, segmentation_dataset_dir, dirs_exist_ok=True)

In [ ]:
# Anotate images for segmentaion task
splits = ["train", "test", "valid"]
for split in splits:
    dataset_split_dir = F"{segmentation_dataset_dir}/{split}"
    
    # Delete detection labels
    shutil.rmtree(f"{dataset_split_dir}/labels")

    auto_annotate(data=f'{dataset_split_dir}/images', det_model=detection_model_config["tuned_model_path"], sam_model=sam_path, output_dir=f'{dataset_split_dir}/labels')

### Fix yaml paths to absolute instead of relative

In [ ]:
with open(segmentation_yaml, "r") as f:
    data = yaml.safe_load(f)

for split, dir in [("train", "train"), ("val", "valid"), ("test", "test")]:
    if split in data:
        data[split] = str(segmentation_dataset_root / dir / "images")

with open(segmentation_yaml, "w") as f:
    yaml.dump(data, f)

## Fine-tune the segmentation model

In [ ]:
# Download pretrained model
model = YOLO(segmentation_model_config["model_name"])

In [ ]:
model.train(
    data=segmentation_yaml, 
    epochs=100, 
    batch=8, 
    project=segmentation_model_config["project_dir"], 
    name=segmentation_model_config["run_name"]
)

In [ ]:
# Save best model in "models" folder
shutil.copy(f"{segmentation_model_config["project_dir"]}/{segmentation_model_config["run_name"]}/weights/best.pt", segmentation_model_config["tuned_model_path"])

## Delete the automatically downloaded pretrained models from disk

In [13]:
os.remove(detection_model_config["model_name"])
os.remove(segmentation_model_config["model_name"])